In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import surp
import arya

In [ ]:
modelname = "perturbations/const_cc"

In [ ]:
walks_raw = pd.read_csv(f"../../models/{modelname}/migration_initial_final.dat")

In [ ]:
dt = 0.02

In [ ]:
walks = walks_raw.set_index(["zone", "i"])

In [ ]:
walks.z_final

In [ ]:
stars = surp.ViceModel.from_file(f"../../models/{modelname}/model.json").stars_unsampled

In [ ]:
stars["t_int"] = np.int64(np.round(stars.formation_time / dt))
stars


In [ ]:
stars["z_final"] = walks.z_final.loc[np.int64(stars.zone_origin), stars.t_int].values

In [ ]:
stars

In [ ]:
np.sum(~np.isnan(stars.z_final))

In [ ]:
walks.z_final.loc[np.array([1,2,3]), np.array([3,4,56])]

## Reproduce figures

In [ ]:
R_bins = [3,5,7,9,11,13]
age_bins = [0,2,4,6,8,10,12]

In [ ]:
stars["R_bin"] = pd.cut(stars.r_origin, R_bins, labels=False)
stars["R_bin_final"] = pd.cut(stars.r_final, R_bins, labels=False)

stars["age_bin"] = pd.cut(stars.formation_time, age_bins, labels=False)

In [ ]:
age_labels = [f"{age_bins[i]} - {age_bins[i+1]}" for i in range(len(age_bins) - 1)]
R_labels = [f"$R_{{\\rm birth}} \\in [{R_bins[i]},\ {R_bins[i+1]})$ kpc" for i in range(len(R_bins) - 1)]
R_labels_final = [f"$R_{{\\rm final}} \\in [{R_bins[i]},\ {R_bins[i+1]})$ kpc" for i in range(len(R_bins) - 1)]

In [ ]:
fig, axs = plt.subplots(1, len(R_bins)-1, figsize=(8, 2), sharey=True)

for i in range(len(R_bins)-1):
    plt.sca(axs[i])
    filt = stars.R_bin == i
    df = stars[filt]
    
    hm = arya.HueMap((0, len(age_labels)))
    for ii in range(len(age_labels) - 1):
        g = df[df.age_bin == ii]
    
        plt.hist(g.r_final, histtype="step", weights=g.mass, density=True, label=age_labels[ii], color=hm(ii), bins=np.linspace(0, 20, 30))

    
    plt.xlabel(r"$R_{\rm final}$ / kpc")
    plt.title(R_labels[i])
    if i == 0:
        plt.ylabel("PDF")
    if i == len(R_bins) - 2:
        arya.Legend(-1, title="Age / Gyr")        



In [ ]:
fig, axs = plt.subplots(1, len(R_bins)-1, figsize=(8, 2), sharey=True)

for i in range(len(R_bins)-1):
    plt.sca(axs[i])
    filt = stars.R_bin_final == i
    df = stars[filt]
    
    hm = arya.HueMap((0, len(age_labels)))
    for ii in range(len(age_labels) - 1):
        g = df[df.age_bin == ii]
    
        plt.hist(g.z_final, histtype="step", weights=g.mass, density=True, label=age_labels[ii], color=hm(ii), bins=np.linspace(-2, 2, 30))

    
    plt.xlabel(r"$z_{\rm final}$ / kpc")
    plt.title(R_labels_final[i])

    if i == 0:
        plt.ylabel("PDF")
        
    if i == len(R_bins) - 2:
        arya.Legend(-1, title="Age")  